### 국회의원 현황정보 수집/분석/시각화/저장
* 이름과 id추출
* 상세페이지 정보 추출(json으로 저장)
* 상세정보를 dataframe으로 변환
* 시각화(막대그래프, 히스토그램, 파이챠트, 히트맵
* table로 저장

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [7]:
url = 'https://assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do'
req_param_dict ={
    'currentPage' : 1,
    'rowPerPage' : 500
}
res = requests.get(url,params=req_param_dict)
print(res.status_code)

member_id_list=[]

if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select('div.memberna_list dl dt a')))
    atag_list = soup.select('div.memberna_list dl dt a')
    for atag in atag_list:
        herf = atag['href']
        matched = re.search(r'(\d+)', herf)
        if matched:
            member_id = matched.group(0)
            member_id_list.append(member_id)
    print(len(member_id_list))
    print(member_id_list)

200
295
295
['9771230', '9771142', '9771174', '9771233', '9771283', '9770933', '9771116', '9771276', '9771168', '9771007', '9771109', '9771180', '9770931', '9771224', '9771165', '9770679', '9771108', '9770977', '9771259', '9770998', '9770927', '9771258', '9770712', '9771152', '9771164', '9771181', '9770719', '9770997', '9771136', '9770723', '9771120', '9771173', '9770941', '9770984', '9771285', '9771268', '9770725', '9770527', '9771024', '9771195', '9770985', '9771208', '9771089', '9771210', '9771220', '9771144', '9771169', '9771111', '9771225', '9770975', '9770981', '9770934', '9771249', '9771186', '9771147', '9771127', '9771214', '9771206', '9771171', '9771288', '9771023', '9771090', '9771009', '9771192', '9770982', '9770986', '9770741', '9771237', '9770742', '9770991', '9771223', '9771270', '9771216', '9771134', '9771228', '9770752', '9770753', '9770755', '9771141', '9771274', '9771091', '9771187', '9771204', '9771175', '9771159', '9770764', '9770911', '9771248', '9770767', '9770768

In [ ]:
import requests
from  bs4 import BeautifulSoup
import  re
from urllib.parse import urljoin

member_detail_list = []
for idx, mem_id in enumerate(member_id_list,1):
